In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# Both MNIST and Fashion-MNIST can be loaded from Keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

from tqdm import tqdm
from tqdm.autonotebook import tqdm, trange

In [ ]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert image pixels to floats between 0 and 1
X_train = x_train / 255
X_test = x_test / 255

# Convert output to one hot encoding
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [ ]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(layers.Input(shape = (28, 28, 1,)))
model.add(layers.Conv2D(32, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D( pool_size = (2, 2)))
model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation = "softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print( model.summary() )

In [ ]:
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True)

In [ ]:
train_data = list(train_dataset)

In [ ]:
train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [ ]:
@tf.function
def validation_step(x_val, y_val):
    val_logits = model(x_val, training=False)
    val_acc_metric.update_state(y_val, val_logits)

In [ ]:
epochs = 50

for epoch in range(epochs):    
    steps = trange(len(train_data), bar_format="{desc}\t{percentage:3.0f}% {r_bar}")
    for i in steps:
        step = i
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        
        loss_value = train_step(x_batch_train, y_batch_train)
        
        steps.set_description("Epoch " + str(epoch+1) + '/' + str(epochs) + "\tLoss: " + str(float(loss_value))[:6]
                              + "\tAccuracy: " + str(float(train_acc_metric.result()))[:6])
        
        if i == len(train_data)-1:
          
            for x_batch_val, y_batch_val in validation_dataset:
                validation_step(x_batch_val, y_batch_val)

            steps.set_description(steps.desc + "\tValidation accuracy: " + str(float(val_acc_metric.result()))[:6])

    train_acc_metric.reset_states()
    val_acc_metric.reset_states()